In [ ]:
# putting the contents of a text file into a panda dataframe

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('death_lessons_dataframe.csv')

In [4]:
files = !ls downloads/

In [5]:
cd downloads

/media/drcjar/homemedia/dropbox/Dropbox/Documents/death-lessons_old/downloads


In [6]:
text_reports = {}

for f in files:
    try:
        with open(f, 'r') as reader:
            text_reports[f] = reader.read()
    except:
        continue

In [7]:
df1 = pd.DataFrame.from_dict(text_reports, orient='index', columns=['text'])
df1 = df1.reset_index()
df1.columns = ['filename', 'text']

In [8]:
# find reference in our dataframe and in our filenames
df['ref'] = df['ref'].str.extract(r'(\d{4}-\d{4})') # extract reference if there is one
df1['ref'] = df1['filename'].str.extract(r'(\d{4}-\d{4})') # extract reference if there is one

In [9]:
df.ref.isnull().value_counts()

False    3054
True      117
Name: ref, dtype: int64

In [10]:
df1.ref.isnull().value_counts()

False    11341
True       707
Name: ref, dtype: int64

In [11]:
# discard nulls for now
df = df[~df.ref.isnull()]
df1 = df1[~df1.ref.isnull()]

In [12]:
# merge on ref
df = pd.merge(df,df1, on='ref')

In [13]:
df.ref.isnull().value_counts()

False    11418
Name: ref, dtype: int64

In [19]:
df

,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category,filename,text
0,linda-phillipson,<p>Date of report: 8 September 2020</p>,2020-0172,<p>Deceased name: Linda Phillipson</p>,<p>Coroner name: Veronica Hamilton-Deeley</p>,<p>Coroner Area: Brighton and Hove</p>,CATEGORY: HOSPITAL DEATH (CLINICAL PROCEDURES ...,Linda-Phillipson-2020-0172_Redacted.pdf.txt,"VERONICA HAMI LT ON -DEELEY DL, \nLLB. \nHer..."
1,peter-howarth,<p>Date of report: 8 September 2020</p>,2020-0171,<p>Deceased name: Peter Howarth</p>,<p>Coroner name: Chris Morris</p>,<p>Coroner Area: Greater Manchester South</p>,CATEGORY: CARE HOME HEALTH RELATED DEATHS; HOS...,Peter-Howarth-2020-0171_Redacted.pdf.txt,\ \n\nREGULATION 28: REPORT TO PREVENT FUTURE...
2,laura-parsons,<p>Date of report: 3 September 2020</p>,2020-0170,<p>Deceased name: Laura Parsons</p>,<p>Coroner name: James Thompson</p>,<p>Coroner Area: County Durham &amp; Darlingto...,CATEGORY: COMMUNITY HEALTHCARE DEATHS,Laura-Parsons-2020-0170-2.pdf.txt,\n\n \n\n1 \n\n2 \n\n3 \n\n4 \n\n \n \n \nREG...
3,ellie-isaacs,<p>Date of report: 7 September 2020</p>,2020-0169,<p>Deceased name: Ellie Isaacs</p>,<p>Coroner name: Nadia Persaud</p>,<p>Coroner Area: East London</p>,CATEGORY: ROAD (HIGHWAYS SAFETY) RELATED SAFETY,Ellie-Issacs-2020-0169.pdf.txt,\n\nEast London Coroners \n\nMISS N PERSAUD ...
4,zoe-knight,<p>Date of report: 4 September 2020</p>,2020-0168,<p>Deceased name: Zoe Knight</p>,<p>Coroner name: Adrian Farrow</p>,<p>Coroner Area: South Manchester</p>,CATEGORY: HOSPITAL DEATH (CLINICAL PROCEDURES ...,Zoe-Knight-2020-0168_Redacted.pdf.txt,REGULATION 28: REPORT TO PREVENT FUTURE DEATH...
...,...,...,...,...,...,...,...,...,...
11413,david-george-white,<p>Date of report: 1 August 2013</p>,2013-0172,<p>Deceased name: David George White</p>,<p>Coroners name: M A Beresford</p>,<p>Coroners Area: South Yorkshire (East)</p>,CATEGORY: ROAD (HIGHWAYS SAFETY) RELATED DEATHS,White-2013-01721.pdf.txt,MS N J MUNDY \nH M CORONER \nSOUTH YORKSHIRE (...
11414,annie-rose-gibson,<p>Date of report: 1 August 2013</p>,2013-0171,<p>Deceased name: Annie Rose Gibson</p>,<p>Coroners name: David Hinchcliff</p>,<p>Coroners Area: West Yorkshire (East)</p>,CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Gibson-2013-01711_page0.txt,HER MAJESTY’S CORONER\nfor the county of West ...
11415,annie-rose-gibson,<p>Date of report: 1 August 2013</p>,2013-0171,<p>Deceased name: Annie Rose Gibson</p>,<p>Coroners name: David Hinchcliff</p>,<p>Coroners Area: West Yorkshire (East)</p>,CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Gibson-2013-01711_page1.txt,a daily basis by her family and home carers fr...
11416,derek-edward-bartlett-twivey,<p>Date of report: 30 July 2013</p>,2013-0175,<p>Deceased name: Derek Edward Bartlett Twivey...,<p>Coroners name: Elisabeth Bussey-Jones</p>,<p>Coroners Area: West Sussex</p>,CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Twivey-2013-0175.pdf.txt,Penelope A Schofield \nSenior Coroner \nCounty...


In [23]:
df.to_json("data.json", lines=True, orient="records")